In [ ]:
# Lower maternal serotonin levels are associated with greater neurodevelopmental severity in autistic children: a partial replication and extension: analytic code

# Setup

## Libraries

In [ ]:
PACKAGES <- rlang::quos(
    chisq.posthoc.test,
    exactRankTests,
    WRS2,
    here,
    broom,
    fs,
    ggplot2,
    gt,
    gtsummary,
    janitor,
    quantreg,
    readxl,
    skimr,
    tidyverse)

lapply(PACKAGES, rlang::quo_name) |> lapply(library, character.only = TRUE) |>
    invisible()

## Paths

In [ ]:
ROOT <- path(
    'data', 
    'raw')

MASTER <- path(
    '0_MASTER',
    '_2022-11-29_UIC_Vanderbilt_de_novo_SNV-InDel_CNV_MASTER_v4_pared_downloaded_20221201.xlsx')

MASTER_SSC_ADOS_CSS <- path(
    '0_MASTER',
    '_2023-04-05_SSC_ADOS_CSS.xlsx')

SSC_DESCRIPTIVES <- path(
    '6_SSC_Version_15_Phenotype_Data_Set',
    'Proband_Data',
    'ssc_core_descriptive.csv')

LCA <- path(
    'SSC_LCA',
    'winsorized_scaled_data_solutions')

LCA_3_CLASS <- path(
    'SSC_3_Classes',
    'SSC_WZ_3Classes.csv')

PGS <- path(
    'ssc_asd_pgs_euro_2022-04-18.csv')

In [ ]:
LCA_5_CLASS <- path(
    'SSC_5_Classes',
    'SSC_WZ_5Classes.csv')

df_5_class <- 
    read_csv(
        here(ROOT, LCA, LCA_5_CLASS),
        col_types = list(Class = col_factor())) |>

    rename(
        Z_nonverbal_iq = nonverbal_iq, 
        Z_ADI_R_a_total = ADI_R_a_total,
        Z_ADI_R_b_non_verbal_total = ADI_R_b_non_verbal_total,
        Z_ADI_R_c_total = ADI_R_c_total,
        Z_ADOS_social_affect_CSS = ADOS_social_affect_CSS,
        Z_ADOS_restricted_repetitive_CSS = ADOS_restricted_repetitive_CSS,
        Z_VABS_composite_standard_score = VABS_composite_standard_score,
        class = Class) |>

    mutate(id = Proband)


In [ ]:
LCA_6_CLASS <- path(
    'SSC_6_Classes',
    'SSC_WZ_6Classes.csv')

df_6_class <- 
    read_csv(
        here(ROOT, LCA, LCA_6_CLASS),
        col_types = list(Class = col_factor())) |>

    rename(
        Z_nonverbal_iq = nonverbal_iq, 
        Z_ADI_R_a_total = ADI_R_a_total,
        Z_ADI_R_b_non_verbal_total = ADI_R_b_non_verbal_total,
        Z_ADI_R_c_total = ADI_R_c_total,
        Z_ADOS_social_affect_CSS = ADOS_social_affect_CSS,
        Z_ADOS_restricted_repetitive_CSS = ADOS_restricted_repetitive_CSS,
        Z_VABS_composite_standard_score = VABS_composite_standard_score,
        class = Class) |>

    mutate(id = Proband)


In [ ]:
LCA_4_CLASS <- path(
    'SSC_4_Classes',
    'SSC_WZ_4Classes.csv')

df_4_class <- 
    read_csv(
        here(ROOT, LCA, LCA_4_CLASS),
        col_types = list(Class = col_factor())) |>

    rename(
        Z_nonverbal_iq = nonverbal_iq, 
        Z_ADI_R_a_total = ADI_R_a_total,
        Z_ADI_R_b_non_verbal_total = ADI_R_b_non_verbal_total,
        Z_ADI_R_c_total = ADI_R_c_total,
        Z_ADOS_social_affect_CSS = ADOS_social_affect_CSS,
        Z_ADOS_restricted_repetitive_CSS = ADOS_restricted_repetitive_CSS,
        Z_VABS_composite_standard_score = VABS_composite_standard_score,
        class = Class) |>

    mutate(id = Proband)


In [ ]:
LCA_2_CLASS <- path(
    'SSC_2_Classes',
    'SSC_WZ_2Classes.csv')

df_2_class <- 
    read_csv(
        here(ROOT, LCA, LCA_2_CLASS),
        col_types = list(Class = col_factor())) |>

    rename(
        Z_nonverbal_iq = nonverbal_iq, 
        Z_ADI_R_a_total = ADI_R_a_total,
        Z_ADI_R_b_non_verbal_total = ADI_R_b_non_verbal_total,
        Z_ADI_R_c_total = ADI_R_c_total,
        Z_ADOS_social_affect_CSS = ADOS_social_affect_CSS,
        Z_ADOS_restricted_repetitive_CSS = ADOS_restricted_repetitive_CSS,
        Z_VABS_composite_standard_score = VABS_composite_standard_score,
        class = Class) |>

    mutate(id = Proband)


In [ ]:
SSC_DIAGNOSIS <- path(
    '6_SSC_Version_15_Phenotype_Data_Set',
    'Proband_Data',
    'ssc_diagnosis.csv')

## Constants

In [ ]:
QUANTILES <- c(0.10, 0.25, 0.50, 0.75, 0.90)

# p value threshold
THRESHOLD <- 2

X_LABELS <- c(
        "0.1%",
        "0.25%",
        "0.50%",
        "0.75%",
        "0.9%")

Y_LABELS <- c(
        "Latino/Latina ethnicity",
        "Asian race",
        "Black race",
        "White race",
        "Male sex",
        "Vanderbilt study site",
        "Maternal whole blood serotonin")

P_ROUND_FACTOR <- 3

# Import data

In [ ]:
df_master <- 
    read_excel(here(ROOT, MASTER), sheet = "Single row per Subject") |>

    rename(
        site = UIC_VAN,
        id = ID_Consolidated,
        relationship = Relationship,
        wb5ht = WB5HT,
        wb5ht_maternal = m_WB5HT,
        wb5ht_paternal = p_WB5HT,
        variant = Include_Variant) |>

    mutate(
        wb5ht_transformed = log10(wb5ht),
        wb5ht_maternal_transformed = log10(wb5ht_maternal))

df_master$site <- as_factor(df_master$site)

df_master$site <- fct_recode(df_master$site,
    "UIC" = "1",
    "Vanderbilt" = "2",
    "Vanderbilt" = "3",
    "UIC" = "4",
    "UIC" = "5")

df_ados <- 
    read_excel(here(ROOT, MASTER_SSC_ADOS_CSS), sheet = "2023-04-05_SSC_ADOS_CSS") |>

    rename(
        site = UIC_VAN, 
        id = SFARI_ID,
        relationship = Rel_to_Proband,
        wb5ht = WB5HT,
        ados_module = ADOS_Module,
        ados_css = ADOS_CSS,
        ados_sa_css = ADOS_SA_CSS,
        ados_rrb_css = ADOS_RRB_CSS) |>

    select(-c(wb5ht, relationship)) |>

    mutate(
        ados_css = na_if(ados_css, 888),
        ados_sa_css = na_if(ados_sa_css, 888),
        ados_rrb_css = na_if(ados_rrb_css, 888))

df_ados$site <- as_factor(df_ados$site)

df_ssc <- df_master |> left_join(df_ados, by = c("site", "id"))

df_descriptives <- 
    read_csv(here(ROOT, SSC_DESCRIPTIVES)) |>
    rename(id = individual)

df_3_class <- 
    read_csv(
        here(ROOT, LCA, LCA_3_CLASS),
        col_types = list(Class = col_factor())) |>

    rename(
        Z_nonverbal_iq = nonverbal_iq, 
        Z_ADI_R_a_total = ADI_R_a_total,
        Z_ADI_R_b_non_verbal_total = ADI_R_b_non_verbal_total,
        Z_ADI_R_c_total = ADI_R_c_total,
        Z_ADOS_social_affect_CSS = ADOS_social_affect_CSS,
        Z_ADOS_restricted_repetitive_CSS = ADOS_restricted_repetitive_CSS,
        Z_VABS_composite_standard_score = VABS_composite_standard_score,
        class = Class) |>

    mutate(id = Proband)

df <- df_ssc |> 
    left_join(df_descriptives, by = "id") |>
    left_join(df_3_class, by = "id") |>

    distinct(id, .keep_all = TRUE) |>

    mutate(
        race_white = if_else(race=="white", 1, 0),
        race_black = if_else(race=="african-amer", 1, 0),
        race_asian = if_else(race=="asian", 1, 0),
        race_other = if_else(race=="other", 1, 0),
        race_more_than_one = if_else(race=="more-than-one-race", 1, 0),
        race_not_specified = if_else(race=="not_specified", 1, 0),
    
        ethnicity_latin = if_else(ethnicity=="hispanic", 1, 0))

df_core <- df |> filter(!is.na(class) & !is.na(wb5ht_maternal))

df_van <- df_core |> filter(site=="Vanderbilt")
df_uic <- df_core |> filter(site=="UIC")

In [ ]:
df_s1 <-df |> filter(!is.na(class))

In [ ]:
df_iq <- 
    read_csv(here(ROOT, SSC_DIAGNOSIS)) |>
    rename(id = individual)


In [ ]:
df_2 <- df_ssc |> 
    left_join(df_descriptives, by = "id") |>
    left_join(df_iq, by = "id") |>
    left_join(df_3_class, by = "id") |>

    distinct(id, .keep_all = TRUE) |>

    mutate(
        race_white = if_else(race=="white", 1, 0),
        race_black = if_else(race=="african-amer", 1, 0),
        race_asian = if_else(race=="asian", 1, 0),
        race_other = if_else(race=="other", 1, 0),
        race_more_than_one = if_else(race=="more-than-one-race", 1, 0),
        race_not_specified = if_else(race=="not_specified", 1, 0),
    
        ethnicity_latin = if_else(ethnicity=="hispanic", 1, 0))

df_core <- df |> filter(!is.na(class) & !is.na(wb5ht_maternal))


#df_core <- df_core |> filter(!is.na(ados_sa_css))

df_van <- df_core |> filter(site=="Vanderbilt")
df_uic <- df_core |> filter(site=="UIC")

In [ ]:
df_lpa <- df_2 |> filter(!is.na(class))

In [ ]:
contingency_table <- df_lpa %>%
  count(class, nonverbal_test) %>%
  pivot_wider(names_from = nonverbal_test, values_from = n, values_fill = list(n = 0))

In [ ]:
contingency_table

In [ ]:
chi_squared_test_result <- chisq.test(contingency_table[,-1]) # Exclude the first column (class) for the test

In [ ]:
print(chi_squared_test_result)

In [ ]:
library(chisq.posthoc.test)

In [ ]:
contingency_table <- table(df_lpa$class, df_lpa$nonverbal_test)

In [ ]:
# Filter for relevant classes and nonverbal_test types (modify as needed for your comparisons)
filtered_data <- df_lpa %>%
  filter(class %in% c("1", "2"), nonverbal_test %in% c("wisc", "wasi"))

# Create a 2x2 contingency table
contingency_table <- table(filtered_data$class, filtered_data$nonverbal_test)

# Perform Fisher's Exact Test
fisher_test_result <- fisher.test(contingency_table)

# View the result
print(fisher_test_result)


In [ ]:
posthoc_result <- chisq.posthoc.test(contingency_table, method = "bonferroni")
print(posthoc_result)

In [ ]:
df_lpa <- df_lpa |> mutate(
    included_in_analysis = if_else(!is.na(wb5ht_maternal), "Maternal serotonin data available", "Maternal serotonin data not available"))

In [ ]:
table_sx <- table_df |> tbl_summary(
    by = class,
    type = list(
        age_years ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age_years,
        site,
        sex,
        race,
        ethnicity),
    label = list(
        age_years ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity ~ "Ethnicity"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants by profile**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1_anonymized_full_lpa.html"))

In [ ]:
df_lpa |> colnames()

# Descriptive stats

In [ ]:
table_df <- df_core |> 
    select(
        age_years,
        class,
        site, 
        sex,
        race,
        ethnicity,
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
        ados_sa_css,
        ados_rrb_css,
        adi_r_soc_a_total,
        adi_r_comm_b_non_verbal_total,
        adi_r_rrb_c_total,
        vineland_ii_composite_standard_score,
        ssc_diagnosis_nonverbal_iq
    ) 

In [ ]:
table_df$class <- recode_factor(table_df$class,
    "1" = "Low severity",
    "2" = "Moderate severity",
    "3" = "High severity")

In [ ]:
table_df$race <- recode_factor(table_df$race, 
    `african-amer` = "Black", 
    white = "White",
    asian = "Asian",
    `more-than-one-race` = "More than one race",
    `not-specified` = "Not specified",
    `other` = "Other")

In [ ]:
table_df$sex <- recode_factor(table_df$sex, 
    male = "Male", 
    female = "Female")

In [ ]:
table_df$ethnicity <- recode_factor(table_df$ethnicity, 
    hispanic = "Latine", 
    `non-hispanic` = "Non-Latine")

In [ ]:
# anonymizing step

table_df$site <- recode_factor(table_df$site, 
    `UIC` = "University X", 
    `Vanderbilt` = "University Y")

In [ ]:
table1 <- table_df |> tbl_summary(
    by = class,
    type = list(
        age_years ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age_years,
        site,
        sex,
        race,
        ethnicity),
    label = list(
        age_years ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity ~ "Ethnicity"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants by profile**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1_anonymized_full_lpa.html"))

In [ ]:
table_df |> group_by(class) |> skim(wb5ht_paternal)

# Maternal WB5-HT by class membership

In [ ]:
df_core <- df_core |> mutate(class_3 = if_else(class == 3, 1, 0))

## Wilcoxon rank sum test

In [ ]:
wilcox.exact(wb5ht_maternal ~ class_3, df_core)

## Harrell-Davis quantile estimator

In [ ]:
class_3_vs_non <- qcomhd(wb5ht_maternal ~ class_3, df_core)

In [ ]:
class_3_vs_non_table <- class_3_vs_non$partable |>
    select(-c(p.crit)) |>
    gt() |>
    tab_header(
        title = "Maternal WB5-HT by quantile for probands in class 3 compared to class 1 or 2") |>
    cols_label(
        q = "QU",
        n1 = md("*N* (Class 1 or 2)"),
        n2 = md("*N* (Class 3)"),
        est1 = "Median (Class 1 or 2)",
        est2 = "Median (Class 3)",
        "est1-est.2" = "Difference",
        ci.low = "CI Lower",
        ci.up = "CI Upper",
        p.value = md("*Adj. p*")) |>
    fmt_number(
        columns = 4:8,
        decimals = 2) |>
    fmt_number(
        columns = 9,
        decimals = 3) |>
    cols_align(
        align = "right",
        columns = everything()) |>
    tab_style(
        style = cell_text(weight = "bold"),
        locations = cells_body(
            rows = p.value < 0.05))

In [ ]:
class_3_vs_non_table |> gtsave("class_3_vs_non.html")

In [ ]:
df_core_fig <- df_core

In [ ]:
df_core_fig$class_3 <- as.factor(df_core_fig$class_3)

In [ ]:
df_core_fig$class <-
    factor(
        df_core_fig$class,
        levels = c("1", "2", "3"))

In [ ]:
ggplot(df_core_fig, aes(x = class, y = wb5ht_maternal)) +
    geom_violin(
        alpha = 0.5, 
        aes(color = class, fill = class),
        scale = "area") +
    geom_violin(
        alpha = 0, 
        aes(),
        draw_quantiles = c(0.2, 0.4, 0.6, 0.8),
        scale = "area") +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
#         plot.title = element_text(size = 12),
         panel.grid.major = element_blank(),
         panel.grid.minor = element_blank()) +
    labs(
        title = "Maternal WB5-HT by proband latent profile",
        y = "Maternal WB5-HT (ng/ml)",
        x = "Profile") +
    scale_fill_brewer(palette="Set1")

In [ ]:
ggsave(here("output","3class.png"))

In [ ]:
library(ggpubr)

In [ ]:
df_class_table <- df_core |> 
    select(class, wb5ht_maternal) |>
    tbl_summary(
        by = class,
        statistic = all_continuous() ~ "{mean} ({sd})",
        label = wb5ht_maternal ~ "Maternal whole blood serotonin (ng/ml)") |>
    add_overall(last = TRUE) |>
    modify_header(
        label = "") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Maternal whole blood serotonin levels by class membership**") |> 
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    add_p()

In [ ]:
df_class_table |> 
    as_gt() |>
    gtsave(here("output","class_table.html"))

# ADOS SA CSS

### Linear model

In [ ]:
model <- lm(
    ados_sa_css ~ 
        wb5ht_maternal + 
        age_at_ados +
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(
    ados_sa_css ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model, conf.int=TRUE)

### Quantile model

In [ ]:
rq_fit <- rq(
    ados_sa_css ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_ados_sa <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < THRESHOLD, `Pr(>|t|)`, NA))

fit_ados_sa$x <- as_factor(fit_ados_sa$x)

In [ ]:
fit_ados_sa_subset <- fit_ados_sa |> filter(predictor == "wb5ht_maternal")

fit_ados_sa_subset <- fit_ados_sa_subset |> mutate(predictor = "ados_sa")

In [ ]:
summary(rq_fit, se="ker")

In [ ]:
fit_ados_sa |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of ADOS SA CSS:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_ados_sa.png"))

# ADOS RRB CSS

### Linear model

In [ ]:
model <- lm(
    ados_rrb_css ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    ados_rrb_css ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_ados_rrb <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < 5, `Pr(>|t|)`, NA))

fit_ados_rrb$x <- as_factor(fit_ados_rrb$x)

In [ ]:
fit_ados_rrb_subset <- fit_ados_rrb |> filter(predictor == "wb5ht_maternal")

fit_ados_rrb_subset <- fit_ados_rrb_subset |> mutate(predictor = "ados_rrb")

In [ ]:
fit_ados_rrb |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of ADOS RRB CSS:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_ados_rrb.png"))

# ADI-R A

### Linear model

In [ ]:
model <- lm(
    adi_r_soc_a_total ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    adi_r_soc_a_total ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_adi_a <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < 1, `Pr(>|t|)`, NA))

fit_adi_a$x <- as_factor(fit_adi_a$x)

In [ ]:
fit_adi_a_subset <- fit_adi_a |> filter(predictor == "wb5ht_maternal")


In [ ]:
fit_adi_a_subset <- fit_adi_a_subset |> mutate(predictor = "adi_a")

In [ ]:
fit_adi_a |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of ADI-R A:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_adi_a.png"))

# ADI-R B

### Linear model

In [ ]:
model <- lm(
    adi_r_comm_b_non_verbal_total ~ 
        wb5ht_maternal_transformed + 
#        age_at_ados +
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    adi_r_comm_b_non_verbal_total ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_adi_b <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < THRESHOLD, `Pr(>|t|)`, NA))

fit_adi_b$x <- as_factor(fit_adi_b$x)

In [ ]:
fit_adi_b_subset <- fit_adi_b |> filter(predictor == "wb5ht_maternal")

fit_adi_b_subset <- fit_adi_b_subset |> mutate(predictor = "adi_b")

In [ ]:
fit_adi_b |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of ADI-R B:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_adi_b.png"))

# ADI-R C

### Linear model

In [ ]:
model <- lm(
    adi_r_rrb_c_total ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    adi_r_rrb_c_total ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_adi_c <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < THRESHOLD, `Pr(>|t|)`, NA))

fit_adi_c$x <- as_factor(fit_adi_c$x)

In [ ]:
fit_adi_c_subset <- fit_adi_c |> filter(predictor == "wb5ht_maternal")

fit_adi_c_subset <- fit_adi_c_subset |> mutate(predictor = "adi_c")

In [ ]:
fit_adi_c |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of ADI-R C:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_adi_c.png"))

# VABS

### Linear model

In [ ]:
model <- lm(
    vineland_ii_composite_standard_score ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    vineland_ii_composite_standard_score ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_vabs <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < THRESHOLD, `Pr(>|t|)`, NA))

fit_vabs$x <- as_factor(fit_vabs$x)

In [ ]:
fit_vabs_subset <- fit_vabs |> filter(predictor == "wb5ht_maternal")

fit_vabs_subset <- fit_vabs_subset |> mutate(predictor = "vabs")

In [ ]:
fit_vabs |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of VABS-II:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_vabs.png"))

# NVIQ

### Linear model

In [ ]:
model <- lm(
    ssc_diagnosis_nonverbal_iq ~ 
        wb5ht_maternal_transformed + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model)

### Quantile model

In [ ]:
rq_fit <- rq(
    ssc_diagnosis_nonverbal_iq ~ 
        wb5ht_maternal + 
        site + 
        sex + 
        race_white + 
        race_black + 
        race_asian + 
        ethnicity_latin, 
    data = df_core, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")

fit_list <- vector("list", 5)

for (x in 1:(length(QUANTILES))) {
    fit_coefficients[[x]][["coefficients"]] <- fit_coefficients[[x]][["coefficients"]] |> 
        cbind(x)

    fit_list[[x]] <- fit_coefficients[[x]][["coefficients"]] |> 
        as_tibble(rownames = NA) |> 
        rownames_to_column(var = "predictor")
}

fit_nviq <- bind_rows(fit_list) |> 
    select(c("predictor","Pr(>|t|)","x")) |> 
    arrange(predictor) |> 
    slice(6:n()) |>
    arrange(x) |>
    mutate(p_thresholded = ifelse(`Pr(>|t|)` < THRESHOLD, `Pr(>|t|)`, NA))

fit_nviq$x <- as_factor(fit_nviq$x)

In [ ]:
fit_nviq_subset <- fit_nviq |> filter(predictor == "wb5ht_maternal")

fit_nviq_subset <- fit_nviq_subset |> mutate(predictor = "nviq")

In [ ]:
fit_nviq_subset |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_y_discrete(labels = Y_LABELS) +
    scale_x_discrete(labels = X_LABELS) +
    scale_fill_continuous(trans = "reverse") +
    xlab("5-quantile") +
    ylab("Predictor") +
    labs(
        title = "Predictors of NVIQ:\n Statistical significance at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave(here("output","heatmap_nviq.png"))

# Visualization across indicators

In [ ]:
fit_subset <- bind_rows(
    fit_ados_sa_subset,
    fit_ados_rrb_subset,
    fit_adi_a_subset,
    fit_adi_b_subset,
    fit_adi_c_subset,
    fit_vabs_subset,
    fit_nviq_subset) |> arrange(x)

In [ ]:
Y_LABELS_SUBSET <- c(
    "ADI-R domain A",
    "ADI-R domain B",
    "ADI-R domain C",
    "ADOS RRB CSS",
    "ADOS SA CSS",
    "NVIQ",
    "VABS-II composite")

In [ ]:
fit_subset |> 
    ggplot(aes(
        x = x, 
        y = predictor, 
        fill = p_thresholded)) +
    geom_tile(
        color = "white",
        lwd = 1.5,
        show.legend = FALSE) +
    geom_text(
        aes(label = round(p_thresholded, P_ROUND_FACTOR)), 
        color = "white") +
    coord_fixed() +
    scale_x_discrete(labels = X_LABELS) +
    scale_y_discrete(labels = Y_LABELS_SUBSET) +
    scale_fill_continuous(trans = "reverse") +
    xlab("Quantile") +
    ylab("LCA indicator variable") +
    labs(
        title = "Statistical significance of maternal WB5-HT\n as predictor of each LCA indicator variable\n at each quantile") +
    theme(
        panel.background = element_blank())

In [ ]:
ggsave("significance.png")

# Sensitivity analyses:

## Restricted by institution

In [ ]:
df_core_uic <- df_core |> filter(site=="UIC")
df_core_van <- df_core |> filter(site=="Vanderbilt")

In [ ]:
df_core_uic$wb5ht_maternal |> mean()

In [ ]:
df_core_van$wb5ht_maternal |> mean()

In [ ]:
class_3_vs_non_uic <- qcomhd(wb5ht_maternal ~ class_3, df_core_uic)

In [ ]:
class_3_vs_non_uic

In [ ]:
class_3_vs_non_van <- qcomhd(wb5ht_maternal ~ class_3, df_core_van)

In [ ]:
class_3_vs_non_van

## Restricted by race

In [ ]:
df_core_white <- df_core |> filter(race_white==1)

In [ ]:
class_3_vs_non_white <- qcomhd(wb5ht_maternal ~ class_3, df_core_white)

In [ ]:
class_3_vs_non_white